## scrapy 실행

In [83]:
import scrapy, requests
from scrapy.http import TextResponse

In [84]:
#!rm -rf covid_19_total
!scrapy startproject covid_19_total

Error: scrapy.cfg already exists in /home/ubuntu/python3/notebook/covid_19/covid_19_total


In [2]:
headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'}
req = requests.get('http://ncov.mohw.go.kr/',headers=headers)
response = TextResponse(req.url, body=req.text, encoding='utf-8')
response

<200 http://ncov.mohw.go.kr/>

In [70]:
date = response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/h2/a/span[1]/text()').extract()
date = list(date)[0].split(" ") 
date = date[0:3]
date = " ".join(date)
date = date.replace("(","21.").replace(",","")
country_in =list(map(int, response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/div[1]/div/ul/li[1]/span[2]/text()').extract()))
country_out =list(map(int, response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/div[1]/div/ul/li[2]/span[2]/text()').extract()))
total_country = [country_in+country_out for country_in,country_out in zip(country_in,country_out)][0]
capital_distance = response.xpath('//*[@id="main_maplayout"]/button[9]/span[2]/text()').extract()[0]
noncapital_distance = response.xpath('//*[@id="main_maplayout"]/button[17]/span[2]/text()').extract()[0]

date,country_in, country_out, total_country,capital_distance,noncapital_distance


('21.3.5. 00시 기준', [381], [17], 398, '2', '1.5')

In [13]:
%%writefile covid_19_total/covid_19_total/items.py
import scrapy


class Covid19TotalItem(scrapy.Item):
    date = scrapy.Field()
    country_in = scrapy.Field()
    country_out = scrapy.Field()
    total_country = scrapy.Field()
    capital_distance = scrapy.Field()
    noncapital_distance = scrapy.Field()

Overwriting covid_19_total/covid_19_total/items.py


In [71]:
%%writefile  covid_19_total/covid_19_total/spiders/spider.py
import scrapy,re
from covid_19_total.items import Covid19TotalItem

class spider(scrapy.Spider):
    name = "covid_19_total"
    start_urls =  ["http://ncov.mohw.go.kr/"]
    
    def parse(self, response):  
#         links = response.xpath('/html/head/meta[8]/@content').extract()
#         for link in links:
            yield scrapy.Request("http://ncov.mohw.go.kr/", callback=self.parse_content)
   
    def parse_content(self,response):
        item = Covid19TotalItem()
        date = response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/h2/a/span[1]/text()').extract()
        date = list(date)[0].split(" ") 
        date = date[0:3]
        date = " ".join(date)
        item['date'] = date.replace("(","21.").replace(",","")
        item['country_in'] =list(map(int, response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/div[1]/div/ul/li[1]/span[2]/text()').extract()))
        item['country_out'] =list(map(int, response.xpath('/html/body/div/div[5]/div[2]/div/div[1]/div[1]/div[1]/div/ul/li[2]/span[2]/text()').extract()))
        item['total_country'] = [item['country_in']+item['country_out'] for item['country_in'],item['country_out'] in zip(item['country_in'],item['country_out'])][0]
        item['capital_distance'] = response.xpath('//*[@id="main_maplayout"]/button[9]/span[2]/text()').extract()[0]
        item['noncapital_distance'] = response.xpath('//*[@id="main_maplayout"]/button[17]/span[2]/text()').extract()[0]
        yield item

Overwriting covid_19_total/covid_19_total/spiders/spider.py


In [72]:
%pwd

'/home/ubuntu/python3/notebook/covid_19'

In [73]:
%%writefile run.sh
cd /home/ubuntu/python3/notebook/covid_19/covid_19_total/
scrapy crawl covid_19_total -o covid_19_total.csv

Overwriting run.sh


In [74]:
!/bin/bash run.sh

2021-03-05 10:58:59 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: covid_19_total)
2021-03-05 10:58:59 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Dec 25 2020, 07:37:56) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-05 10:58:59 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-05 10:58:59 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'covid_19_total',
 'NEWSPIDER_MODULE': 'covid_19_total.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['covid_19_total.spiders']}
2021-03-05 10:58:59 [scrapy.extensions.telnet] INFO: Telnet Password: 1bb1d494a2498cac
2021-03-05 10:58:59 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.m

In [75]:
import pandas as pd
df = pd.read_csv('./covid_19_total/covid_19_total.csv')
df

,capital_distance,country_in,country_out,date,noncapital_distance,total_country
0,2,426,18,"(3.3. 00시 기준, '20.1.3. 이후 누계)",1.5,444
1,capital_distance,country_in,country_out,date,noncapital_distance,total_country
2,2,426,18,"(3.3. 00시 기준, '20.1.3. 이후 누계)",1.5,444
3,capital_distance,country_in,country_out,date,noncapital_distance,total_country
4,2,401,23,"(3.4. 00시 기준, '20.1.3. 이후 누계)",1.5,424
5,capital_distance,country_in,country_out,date,noncapital_distance,total_country
6,2,381,17,3.5. 00시 기준,1.5,398
7,capital_distance,country_in,country_out,date,noncapital_distance,total_country
8,2,381,17,3.5. 00시 기준,1.5,398
9,capital_distance,country_in,country_out,date,noncapital_distance,total_country


## pymongo 실행

In [76]:
import pymongo

In [85]:
client = pymongo.MongoClient("mongodb://")
collection = client.covid19_total.total

In [86]:
data = {"title": "count"}
collection.insert(data)

ObjectId('604192f66e65021c657dc81f')

In [87]:
%%writefile covid_19_total/covid_19_total/mongodb.py
import pymongo

client = pymongo.MongoClient("mongodb://")
collection = client.covid19_total.total

Overwriting covid_19_total/covid_19_total/mongodb.py


In [89]:
%%writefile covid_19_total/covid_19_total/pipelines.py
from itemadapter import ItemAdapter
from .mongodb import collection

class Covid19TotalPipeline:
    def process_item(self, item, spider):
        data = {"date": item['date'], "country_in": item['country_in'],
                "country_out": item['country_out'], "total_country": item['total_country'],
                "capital_distance": item['capital_distance'], "noncapital_distance": item['noncapital_distance']}
        collection.insert(data)
        return item
               

Overwriting covid_19_total/covid_19_total/pipelines.py


In [1]:
!/bin/bash run.sh

2021-03-05 11:21:24 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: covid_19_total)
2021-03-05 11:21:24 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Dec 25 2020, 07:37:56) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-05 11:21:24 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-05 11:21:24 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'covid_19_total',
 'NEWSPIDER_MODULE': 'covid_19_total.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['covid_19_total.spiders']}
2021-03-05 11:21:24 [scrapy.extensions.telnet] INFO: Telnet Password: fe5c6fbcc1138ded
2021-03-05 11:21:24 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.m